In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    
    
    
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install ../input/pyphen/Pyphen-0.10.0-py3-none-any.whl

In [ ]:
!pip install ../input/textstat/textstat-0.7.1-py3-none-any.whl

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import re
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
import textstat
from textstat.textstat import textstatistics
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.decomposition import TruncatedSVD
from xgboost import XGBRegressor
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

In [ ]:
train_df = pd.read_csv('/kaggle/input/commonlitreadabilityprize/train.csv')
test_df = pd.read_csv('/kaggle/input/commonlitreadabilityprize/test.csv')

In [ ]:
train_df.head()

In [ ]:
train_df.shape

In [ ]:
train_df.info()

In [ ]:
train_df.isnull().sum()

In [ ]:
test_df.head()

In [ ]:
test_df.shape

In [ ]:
test_df.info()

In [ ]:
test_df.isnull().sum()

In [ ]:
train_df = train_df.drop_duplicates()
train_df.shape

In [ ]:
test_df = test_df.drop_duplicates()
test_df.shape

In [ ]:
fig = plt.figure(figsize=(10,15))
plt.subplot(2,1,1)
sns.distplot(train_df['target'])
plt.subplot(2,1,2)
sns.distplot(train_df['standard_error'])

The graph representing the target values seems normally distributed representing that rating done for the article where most of them are medium complex and other are either too easy or too hard to be classified correctly.

Graph representing standard_error in the dataset looks to be at 0.5 error value which means that most of the passage which the user is not to understand are misjudge to be of medium complexibility by them.

In [ ]:
#dropping columns url_legal and license
train_df = train_df.drop(columns=['url_legal','license'],axis=1)

Adding few new columns into the train_df displaying summary count of words and sentence in the passage mentioned in excerpt.

In [ ]:
summary_df = train_df

In [ ]:
summary_df.head()

In [ ]:
no_sentence = []
total_word = 0
average_word = 0
no_words = []
average_word_sen = []
for passage in summary_df['excerpt']:
    count_sen = len(re.findall(r'\.', passage))
    no_sentence.append(count_sen)
    for sen in passage:
        count_word = len(re.findall(r'\w+', sen))
        total_word += count_word
    average_word = total_word/count_sen
    no_words.append(total_word)
    average_word_sen.append(average_word)
summary_df['no_sen'] = no_sentence
summary_df['no_word'] = no_words
summary_df['average_word_sen'] = average_word_sen

In [ ]:
count_letter = 0
total_letter = []
all_word = []
count_unique_word = []
for passage in summary_df['excerpt']:
    for word in re.findall(r'\w+', passage):
        all_word.append(word)
        count_letter += len(word)
    all_word = list(set(all_word))
    count_unique_word.append(len(all_word))
    total_letter.append(count_letter)
summary_df['letter_count'] = count_letter
summary_df['count_unique_word'] = count_unique_word

In [ ]:
count = 0
unique_words = []
count_non_stop = []
stop_words = list(set(stopwords.words('english')))
for passage in summary_df['excerpt']:
    words = word_tokenize(passage)
    for w in words:
        if w not in stop_words and w not in unique_words:
            count += 1
            unique_words.append(w)
    count_non_stop.append(count)
summary_df['count_non_stop'] = count_non_stop

In [ ]:
passage_length = []
max_pass_len = []
min_pass_len = []
sent_passage = []
for passage in summary_df['excerpt']:
    passage_length.append(len(passage))
    sent_length = sent_tokenize(passage)
    sent_passage.append(sent_length)
    max_pass_len.append(len(max(sent_length)))
    min_pass_len.append(len(min(sent_length)))
summary_df['passage_length'] = passage_length
summary_df['max_pass_len'] = max_pass_len
summary_df['min_pass_len'] = min_pass_len

In [ ]:
summary_df.info()

In [ ]:
summary_df.head()

# Dale Chall Readability Score

In [ ]:
dale_chall_list = []
for text in summary_df['excerpt']:
    dale_chall = textstat.dale_chall_readability_score(text)
    dale_chall_list.append(dale_chall)
print(len(dale_chall_list))

In [ ]:
coleman_liau_list = []
for text in summary_df['excerpt']:
    coleman_liau = textstat.coleman_liau_index(text)
    coleman_liau_list.append(coleman_liau)
print(len(coleman_liau_list))

In [ ]:
smog_index_list = []
for text in summary_df['excerpt']:
    smog_index = textstat.smog_index(text)
    smog_index_list.append(smog_index)
print(len(smog_index_list))

In [ ]:
automated_readability_index_list = []
for text in summary_df['excerpt']:
    automated_readability_index = textstat.automated_readability_index(text)
    automated_readability_index_list.append(automated_readability_index)
print(len(automated_readability_index_list))

In [ ]:
linsear_write_formula_list = []
for text in summary_df['excerpt']:
    linsear_write_formula = textstat.linsear_write_formula(text)
    linsear_write_formula_list.append(linsear_write_formula)
print(len(linsear_write_formula_list))

In [ ]:
gunning_fog_list = []
for text in summary_df['excerpt']:
    gunning_fog = textstat.gunning_fog(text)
    gunning_fog_list.append(gunning_fog)
print(len(gunning_fog_list))

In [ ]:
flesch_reading_ease_list = []
for text in summary_df['excerpt']:
    flesch_reading_ease = textstat.flesch_reading_ease(text)
    flesch_reading_ease_list.append(flesch_reading_ease)
print(len(flesch_reading_ease_list))

In [ ]:
flesch_kincaid_grade_list = []
for text in summary_df['excerpt']:
    flesch_kincaid_grade = textstat.flesch_kincaid_grade(text)
    flesch_kincaid_grade_list.append(flesch_kincaid_grade)
print(len(flesch_kincaid_grade_list))

In [ ]:
difficult_words_list = []
for text in summary_df['excerpt']:
    difficult_words = textstat.difficult_words(text)
    difficult_words_list.append(difficult_words)
print(len(difficult_words_list))

In [ ]:
summary_df['dale_chall_list'] = dale_chall_list
summary_df['coleman_liau_list'] = coleman_liau_list
summary_df['smog_index_list'] = smog_index_list
summary_df['automated_readability_index_list'] = automated_readability_index_list
summary_df['linsear_write_formula_list'] = linsear_write_formula_list
summary_df ['gunning_fog_list'] = gunning_fog_list
summary_df['flesch_reading_ease_list'] = flesch_reading_ease_list
summary_df['flesch_kincaid_grade_list'] = flesch_kincaid_grade_list
summary_df['flesch_kincaid_grade_list'] = flesch_kincaid_grade_list

In [ ]:
summary_df.info()

In [ ]:
fig = plt.figure(figsize=(20,20))
sns.heatmap(summary_df.corr(), annot = True)

In [ ]:
summary_df = summary_df.drop(columns=['no_sen','no_word', 'dale_chall_list', 'smog_index_list', 'count_non_stop', 'letter_count', 'flesch_reading_ease_list', 'automated_readability_index_list', 'gunning_fog_list', 'flesch_kincaid_grade_list'])

In [ ]:
fig = plt.figure(figsize=(20,20))
sns.heatmap(summary_df.corr(), annot = True)

# Model Building

In [ ]:
def tokenize_def(text):
    text = re.sub(r'[^\w\s]','',text)
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [ ]:
#kf = KFold(n_splits=10, random_state= 42, shuffle= True)
vect = CountVectorizer(tokenizer=tokenize_def)
tfidf = TfidfTransformer()
#clf = RandomForestRegressor()
lgb = LGBMRegressor(max_depth = 2, num_leaves = 50, learning_rate = 0.05)
#xgb = XGBRegressor()
svd = TruncatedSVD(n_components=10, n_iter=10, random_state=42)
#lasso = Lasso()
#ridge = Ridge()

In [ ]:
summary_df_vect = vect.fit_transform(summary_df['excerpt'])

In [ ]:
summary_df_tfidf = tfidf.fit_transform(summary_df_vect)

In [ ]:
summary_df_svd = svd.fit_transform(summary_df_tfidf)

In [ ]:
summary_df_svd = pd.DataFrame(summary_df_svd)

In [ ]:
summary_df_svd.head()

In [ ]:
sample = pd.merge(summary_df, summary_df_svd, left_index=True, right_index=True)

In [ ]:
sample.head(3)

In [ ]:
X = sample.drop(columns=['id', 'excerpt', 'target'], axis=1)
y = sample['target']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
params = {'max_depth':[2],'num_leaves':[50],'learning_rate':[0.05]}
model = GridSearchCV(lgb, params)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
model.score(X_train, y_train)

In [ ]:
model.score(X_test, y_test)

In [ ]:
pred = model.predict(X_test)

In [ ]:
mean_squared_error(y_test, pred, squared=False)

In [ ]:
test_df = test_df.drop(columns=['url_legal','license'],axis=1)

summary_df = test_df

no_sentence = []
total_word = 0
average_word = 0
no_words = []
average_word_sen = []
count_letter = 0
total_letter = []
all_word = []
count_unique_word = []
count = 0
unique_words = []
count_non_stop = []
passage_length = []
max_pass_len = []
min_pass_len = []
sent_passage = []

for passage in summary_df['excerpt']:
    count_sen = len(re.findall(r'\.', passage))
    no_sentence.append(count_sen)
    for sen in passage:
        count_word = len(re.findall(r'\w+', sen))
        total_word += count_word
    average_word = total_word/count_sen
    no_words.append(total_word)
    average_word_sen.append(average_word)
summary_df['no_sen'] = no_sentence
summary_df['no_word'] = no_words
summary_df['average_word_sen'] = average_word_sen


for passage in summary_df['excerpt']:
    for word in re.findall(r'\w+', passage):
        all_word.append(word)
        count_letter += len(word)
    all_word = list(set(all_word))
    count_unique_word.append(len(all_word))
    total_letter.append(count_letter)
summary_df['letter_count'] = count_letter
summary_df['count_unique_word'] = count_unique_word

stop_words = list(set(stopwords.words('english')))
for passage in summary_df['excerpt']:
    words = word_tokenize(passage)
    for w in words:
        if w not in stop_words and w not in unique_words:
            count += 1
            unique_words.append(w)
    count_non_stop.append(count)
summary_df['count_non_stop'] = count_non_stop


for passage in summary_df['excerpt']:
    passage_length.append(len(passage))
    sent_length = sent_tokenize(passage)
    sent_passage.append(sent_length)
    max_pass_len.append(len(max(sent_length)))
    min_pass_len.append(len(min(sent_length)))
summary_df['passage_length'] = passage_length
summary_df['max_pass_len'] = max_pass_len
summary_df['min_pass_len'] = min_pass_len

coleman_liau_list = []
for text in summary_df['excerpt']:
    coleman_liau = textstat.coleman_liau_index(text)
    coleman_liau_list.append(coleman_liau)
    
linsear_write_formula_list = []
for text in summary_df['excerpt']:
    linsear_write_formula = textstat.linsear_write_formula(text)
    linsear_write_formula_list.append(linsear_write_formula)
    
summary_df['coleman_liau_list'] = coleman_liau_list
summary_df['linsear_write_formula_list'] = linsear_write_formula_list

summary_df_vect = vect.fit_transform(summary_df['excerpt'])
summary_df_tfidf = tfidf.fit_transform(summary_df_vect)
summary_df_svd = svd.fit_transform(summary_df_tfidf)
summary_df_svd = pd.DataFrame(summary_df_svd)

test_df = pd.merge(summary_df, summary_df_svd, left_index=True, right_index=True)

df_test = test_df.drop(columns=['id', 'excerpt'], axis=1)

In [ ]:
pred = model.predict(df_test)

In [ ]:
predictions = pd.DataFrame(pred, columns=['target'])
predictions

In [ ]:
submission = test_df[['id']]
submission

In [ ]:
submission = pd.merge(submission,predictions, right_index=True, left_index=True)

In [ ]:
submission

In [ ]:
submission.to_csv("./submission.csv",index=False, float_format='%.1f')